<a href="https://colab.research.google.com/github/takada-at/sep_crawl/blob/master/sep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

クローラーの準備

In [0]:
# @title クローラーの準備
% cd /content
! if [ ! -d sep_crawl ]; then git clone https://github.com/takada-at/sep_crawl.git; fi
% cd sep_crawl
! git pull
! pip install -r requirements-colab.txt

In [0]:
# @title SEP記事のダウンロード
# @markdown 80分程度かかります。
% cd /content/sep_crawl/sep/
! if [ -d ../data/sep ]; then rm -r ../data/sep; fi
! time scrapy crawl entry --loglevel=INFO
! cat ../data/sep/text/*.txt > ../data/sep/sep-entries.txt

In [0]:
# @title ダウンロードしたファイルの確認
# 行数
! wc -l /content/sep_crawl/data/sep/sep-entries.txt 
! head /content/sep_crawl/data/sep/sep-entries.txt 
# ダウンロードした記事の確認
! ls /content/sep_crawl/data/sep/text | head
! ls /content/sep_crawl/data/sep/text | tail
! ls /content/sep_crawl/data/sep/text | wc -l

In [0]:
# @title Google Driveにバックアップ
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

% cd /content
! if [ -d sep-entries ]; then rm -r sep-entries; fi
! mkdir sep-entries
! if [ -f sep-entries.zip ]; then rm sep-entries.zip; fi
! cp /content/sep_crawl/data/sep/sep-entries.txt sep-entries/
! zip /content/sep-entries.zip sep-entries/*
! cp /content/sep-entries.zip "/content/drive/My Drive/"

In [0]:
# @title Google Driveからインポート

from google.colab import drive
drive.mount('/content/drive',  force_remount=True)
! cp "/content/drive/My Drive/sep-entries.zip" /content/sep-entries.zip 
% cd /content
! unzip sep-entries.zip
! mv sep-entries/sep-entries.txt /content/sep_crawl/data/sep/sep-entries.txt

In [0]:
# @title word2vecモデルの作成
from datetime import datetime
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec

sentences = LineSentence('/content/sep_crawl/data/sep/sep-entries.txt')
s = datetime.now()
w2v = Word2Vec(sentences, size=200)
e = datetime.now()
print('create word2vec model: {}'.format(e - s))

In [0]:
# @title 単語ベクトルの確認
print(len(w2v.wv.vocab))
words = np.array(tuple(w2v.wv.vocab.keys()))
print(w2v.wv.most_similar_cosmul(positive=["kant"], topn=3))
# [('hegel', 0.7388843894004822),
# ('hume', 0.7110885381698608),
# ('spinoza', 0.6922929286956787)]

print(w2v.wv.most_similar_cosmul(positive=["berlin", "france"], negative=['germany'], topn=1))

In [0]:
# @title 可視化
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm

topics = ['life', 'universe', 'everything', 'science', 'metaphysics']


dim = 200
neighbor_name_list = []
neighbor_vectors = []
for i, t in enumerate(topics):
  neighbors = w2v.wv.most_similar(positive=[t], topn=20)
  names = [t] + [n[0] for n in neighbors]
  neighbor_name_list.append(names)
  vectors = np.vstack([w2v.wv[w] for w in names])
  neighbor_vectors.append(vectors)

from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=40.0)
matrix = np.vstack(neighbor_vectors)
v2d = tsne.fit_transform(matrix)
x = v2d[:, 0]
y = v2d[:, 1]
plt.figure(figsize=(10, 10))
colors = cm.rainbow(np.linspace(0, 1, len(topics)))
for i, label in enumerate(topics):
  color = colors[i]
  names = neighbor_name_list[i]
  cnt = len(names)
  l = cnt * i
  r = l + cnt
  xs = x[l:r]
  ys = y[l:r]
  plt.scatter(xs, ys, c=[color]*cnt, label=label)
  for j, n in enumerate(names[:10]):
    plt.annotate(n, alpha=0.5, xy=(xs[j], ys[j]))
plt.legend(loc=4)
plt.grid(True)  
plt.show()
  